# Python Looking Under the Hood 
**Julie**

## Brief Summary of Fluent python Study 

<img src="https://img.ridicdn.net/cover/2409002736/xxlarge" style="width:80px; height:100px;" align="left"/>


      - From Sep. 2019 To Jan 2010 (5 months) 
      - a day a weekday
      - 21 Chapters (almost 600 pages) 


- schedule table 

<img src="fluentpyschedule.png" style="width:400px;" >


- We summarized the study in our each ways.


### Part1 Prologue
#### CH1.The Python Data Model

* 데이터 모델 : 일종의 프레임워크, 언어 자체의 구성단위에 대한 인터페이스를 공식적으로 정의한다. 


* Difference between __str__ and __repr__?

https://stackoverflow.com/questions/1436703/difference-between-str-and-repr

* Data Model reference 

https://docs.python.org/3/reference/datamodel.html


* python data model 이해하기

https://www.slideshare.net/dahlmoon/numpy-20160330

* 파이썬의 메타클래스란 무엇인가?

https://code.tutsplus.com/ko/tutorials/quick-tip-what-is-a-metaclass-in-python--cms-26016

https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/

* why python is slow

http://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/



# Why Python is Slow: Looking Under the Hood

http://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/

### what is cpython
https://stackoverflow.com/questions/17130975/python-vs-cpython

<img src=https://i.imgur.com/PJME67T.png style="width:550px">

source : https://indianpythonista.wordpress.com/2018/01/04/how-python-runs/

using `dis` 

In Python, the dis module allows disassembly of Python code into the individual instructions executed by the Python interpreter (usually cPython) for each line. 

In [3]:
import dis
dis.dis('s[a] += b')

  1           0 LOAD_NAME                0 (s)
              2 LOAD_NAME                1 (a)
              4 DUP_TOP_TWO
              6 BINARY_SUBSCR
              8 LOAD_NAME                2 (b)
             10 INPLACE_ADD
             12 ROT_THREE
             14 STORE_SUBSCR
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


## Why Python is Slow

### reason 1. Python is Dynamically Typed rather than Statically Typed

http://jakevdp.github.io/blog/2014/05/09/why-python-is-slow/#1.-Python-is-Dynamically-Typed-rather-than-Statically-Typed.

```c
/* C code */
int a = 1;
int b = 2;
int c = a + b;
```

```python
# python code
a = 1
b = 2
c = a + b
```

#### C Code 

1. Assign <int> 1 to a
2. Assign <int> 2 to b
3. call binary_add<int, int>(a, b)
4. Assign the result to c
    
#### Python Code    
    
1. Assign 1 to a

    - 1a. Set a->PyObject_HEAD->typecode to integer
    - 1b. Set a->val = 1
    

2. Assign 2 to b

    - 2a. Set b->PyObject_HEAD->typecode to integer<br>
    - 2b. Set b->val = 2
    

3. call binary_add(a, b)

    - 3a. find typecode in a->PyObject_HEAD <br>
    - 3b. a is an integer; value is a->val<br>
    - 3c. find typecode in b->PyObject_HEAD<br>
    - 3d. b is an integer; value is b->val<br>
    - 3e. call binary_add<int, int>(a->val, b->val)<br>
    - 3f. result of this is result, and is an integer.
    

4. Create a Python object c

    - 4a. set c->PyObject_HEAD->typecode to integer
    - 4b. set c->val to result

## reason 2. Python is interpreted rather than compiled

## reason 3. Python's object model can lead to inefficient memory access

<img src=http://jakevdp.github.io/images/array_vs_list.png style="width:550px"/>

## Digging into Python Integers


In [1]:
import sys
print(sys.version[:5])

3.6.8


In [2]:
x = 42
x

42

In [3]:
import ctypes

 
https://hg.python.org/cpython/file/3.4/Include/object.h#l105

```c
struct _longobject{
    long ob_refcnt;
    PyTypeObject *ob_type;
    size_t ob_size;
    long ob_digit[1];
};```

In [1]:
class IntStruct(ctypes.Structure):
    _fields_ = [("ob_ref", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong),
                ("ob_digit", ctypes.c_long),
               ]
    
    def __repr__(self):
        return ("IntStruct(ob_digit={self.ob_digit}, refcount={self.ob_ref})").format(self=self)

NameError: name 'ctypes' is not defined

In [ ]:
num = 42
IntStruct.from_address(id(42))

In [ ]:
id(42)

In [ ]:
Istruct = IntStruct.from_address(id(42))
Istruct.ob_type, Istruct.ob_size, Istruct.ob_digit

In [ ]:
type(Istruct)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import sys
plt.loglog(range(1000), [sys.getrefcount(i) for i in range(1000)])
plt.xlabel('integer value')
plt.ylabel('reference count')

## Digging into Python Lists

Let's apply the above ideas to a more complicated type: Python lists. 

Analogously to integers, we find the definition of the list object itself in [Include/listobject.h](https://hg.python.org/cpython/file/3.4/Include/listobject.h#l23)

```c
typedef struct{
    long ob_refcnt;
    PyTypeObject *ob_type;
    Pyssize_t ob_size;
    PyObject **ob_item;
    long allocated;    
}PyListObject;
```

In [ ]:
class ListStruct(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_size", ctypes.c_ulong), 
                ("ob_item", ctypes.c_long),# pyObject** pointer cast to long
                ("ob_digit", ctypes.c_long),
               ]
    
    def __repr__(self):
        return ("ListStruct(len={self.ob_size}, refcount={self.ob_refcnt})").format(self=self)

In [ ]:
L = [1,2,3,4,5]
ListStruct.from_address(id(L))

In [ ]:
tup = [L, L]
ListStruct.from_address(id(L))

In [ ]:
del L[4]

In [ ]:
listStruct = ListStruct.from_address(id(L))
listStruct

In [ ]:
tup[1]

In [ ]:
# get a raw pointer to our list
Lstruct = ListStruct.from_address(id(L))

# create a type which is an array of integer pointers the same length as L
PtrArray = Lstruct.ob_size * ctypes.POINTER(IntStruct)

# instantiate this type using the ob_item pointer
L_values = PtrArray.from_address(Lstruct.ob_item)

## Numpy List

https://github.com/numpy/numpy/blob/maintenance/1.8.x/numpy/core/include/numpy/ndarraytypes.h#L646

In [ ]:
import numpy as np
np.__version__

In [ ]:
class NumpyStruct(ctypes.Structure):
    _fields_ = [("ob_refcnt", ctypes.c_long),
                ("ob_type", ctypes.c_void_p),
                ("ob_data", ctypes.c_long), # char* pointer cas to long
                ("ob_ndim", ctypes.c_int),
                ("ob_shape", ctypes.c_voidp),
                ("ob_strides", ctypes.c_voidp),
               ]
    @property
    def shape(self):
        return tuple((self.ob_ndim*ctypes.c_int64).from_address(self.ob_shape))
    
    @property
    def strides(self):
        return tuple((self.ob_ndim*ctypes.c_int64).from_address(self.ob_strides))
    
    
    def __repr__(self):
        return ("NumpyStruct(shape={self.shape}, refcount={self.ob_refcnt})").format(self=self)

In [ ]:
x = np.random.random((10, 20))
xstruct = NumpyStruct.from_address(id(x))
xstruct

In [ ]:
L = [x,x,x]  # add three more references to x
xstruct

In [ ]:
x = np.arange(10)
xstruct = NumpyStruct.from_address(id(x))
size = np.prod(xstruct.shape)

# assume an array of integers
arraytype = size * ctypes.c_long
data = arraytype.from_address(xstruct.ob_data)

[d for d in data]

In [ ]:
[id(d) for d in data]

and then, what is Data model? 

https://www.slideshare.net/dahlmoon/numpy-20160330
    
data model 이란 python 이 제공하는 하나의 framwork 로 sqeunce, iteration 등의 객체를 말한다. 

In [26]:
type(object)

type

VISUALIZE CODE

http://www.pythontutor.com/visualize.html

http://pythontutor.com/